<a href="https://colab.research.google.com/github/LamineGith/NLP/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Prepare the data
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=20000)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=200)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=200)

# Define the LSTM model
model = keras.Sequential()
model.add(keras.layers.Embedding(20000, 128))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model = model.to(device)
# Fit the model on the training data
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy: ", test_acc)


17464789/17464789 [==============================] - 0s 0us/step
Epoch 1/10
782/782 [==============================] - 506s 643ms/step - loss: 0.4389 - accuracy: 0.7973 - val_loss: 0.3639 - val_accuracy: 0.8466
Epoch 2/10
782/782 [==============================] - 548s 701ms/step - loss: 0.2517 - accuracy: 0.8996 - val_loss: 0.3339 - val_accuracy: 0.8698
Epoch 3/10
782/782 [==============================] - 505s 646ms/step - loss: 0.1712 - accuracy: 0.9372 - val_loss: 0.3625 - val_accuracy: 0.8647
Epoch 4/10
782/782 [==============================] - 505s 646ms/step - loss: 0.1050 - accuracy: 0.9632 - val_loss: 0.4409 - val_accuracy: 0.8578
Epoch 5/10
782/782 [==============================] - 505s 646ms/step - loss: 0.0677 - accuracy: 0.9767 - val_loss: 0.8080 - val_accuracy: 0.8062
Epoch 6/10
782/782 [==============================] - 504s 645ms/step - loss: 0.0528 - accuracy: 0.9818 - val_loss: 0.6311 - val_accuracy: 0.8563
Epoch 7/10
782/782 [==============================] - 504s 

In [11]:
model.predict(x_test)

782/782 [==============================] - 58s 74ms/step


array([[7.2239530e-01],
       [9.9999940e-01],
       [5.0369859e-02],
       ...,
       [3.9283565e-04],
       [4.0250930e-01],
       [9.9993992e-01]], dtype=float32)

In [12]:
!pip install -U torch==1.8.0 torchtext==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data

# Prepare the data
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

text = data.Field(tokenize = 'spacy',batch_first=True)
label = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(text, label)

text.build_vocab(train_data, max_size = 20000, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
label.build_vocab(train_data)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, test_data), batch_size = 64, sort_key = lambda x: len(x.text), sort_within_batch = True)

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        #x = [sent len, batch size]
        embedded = self.dropout(self.embedding(x))
        #embedded = [sent len, batch size, emb dim]
        output, (hidden, cell) = self.lstm(embedded)
        #output = [sent len, batch size, hid dim * num directions]
        #hidden/cell = [num layers * num directions, batch size, hid dim]
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]
        return self.fc(hidden)

# instantiate the model
INPUT_DIM = len(text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

# Define the optimizer and criterion
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

ImportError: ignored

In [17]:
#Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

#Train the model
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
       optimizer.zero_grad()
       predictions = model(batch.text).squeeze(1)
       loss = criterion(predictions, batch.label)
       acc = binary_accuracy(predictions, batch.label)
       loss.backward()
       optimizer.step()
       epoch_loss += loss.item()
       epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

#Evaluate the model
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
      for batch in iterator:
          predictions = model(batch.text).squeeze(1)
          loss = criterion(predictions, batch.label)
          acc = binary_accuracy(predictions, batch.label)
          epoch_loss += loss.item()
          epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

#Define a helper function for binary accuracy
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

#Train the model for a number of epochs
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc100:.2f}%, Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

AttributeError: ignored